---
title: "2.7 Case Study: Network Flows Revisited"
subject: Vector Spaces and Bases
subtitle: linear algebra in electric circuits
short_title: "2.7 Case Study: Network Flows"
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: nodes, edges, Kirchoff's law
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/01_Ch_2_Vector_Spaces_and_Bases/037-network.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 04 - The Fundamental Matrix Subspaces (Kernel, Image, CoKernel, CoImage), Fundamental Theorem of Linear Algebra, and a brief interlude on the Matrix Transpose.pdf>`

## Reading

Material related to this page, as well as additional exercises, can be found in ALA Ch. 1.6.

## Learning Objectives

By the end of this page, you should know:
- how to represent flows in networks as matrices
- the sources and sinks in a network
- the relationship of flow conservation to linear systems

## Incidence Matrix

Consider the directed graph below with $4$ nodes and $5$ edges

:::{figure}../figures/03-network.jpg
:label:network
:alt: Network
:width: 350px
:align: center
:::

We can associate an incidence matrix $A$ with this [graph](#network). Each row corresponds to a node, and each column to an edge, with
\begin{equation}
\label{inc}
a_{ij} =
\begin{cases}
1 & \textrm{if edge} \ j \ \textrm{points \textbf{to} node} \ i  \\
-1 & \textrm{if edge} \ j \ \textrm{points \textbf{from} node} \ i 
\end{cases}.
\end{equation}
For [](#network), $A \in \mathbb{R}^{4 \times 5}$, and
\begin{equation}
\label{inc_A}
A &= \bm 
-1 & -1 & 0 & 0 & 0 \\
1 & 0 & -1 & -1 & 0 \\
0 & 1 & 1 & 0 & -1 \\
0 & 0 & 0 & 1 & 1 
\em
\end{equation}

By considering the four fundamental subspaces (Row, Column, Null and Left Null Space) of $A$, we can completely understand the properties of our network flow problem. The incidence flow can be represented in code using numpy as follows:

In [37]:
import numpy as np
A = np.array([
    [-1, -1, 0, 0, 0],
    [1, 0, -1, -1, 0],
    [0, 1, 1, 0, -1],
    [0, 0, 0, 1, 1]
])

## The Current Law

First, we define the source vector $\vv s = \bm s \\ 0 \\ 0 \\ -s \em$, which captures external flows entering (positive entries) the network referred to as _sources_, and flows leaving (negative entries) the network known as _sinks_. We make sure $\vv 1^{\top} \vv s = 0$. 

The flow conservation equations say that flows entering a node must equal flows leaving a node, which can be written as
\begin{equation}
\label{flow_conserve}
A \vv f + \vv s = \vv 0 \ \textrm{or} \ A \vv f = - \vv s, 
\end{equation}
where $\vv f = \bm f_1 \\ f_2 \\ f_3 \\ f_4 \\ f_5 \em \in \mathbb{R}^{5}$ is the vector of edge flows. The solution set to [](#flow_conserve) characterizes all flows compatible with the network and the source vector $\vv s$.

From [this theorem](./035-kernel_image.ipynb#soln_thm), we see that $A \vv f = - \vv s$ has a solution if and only if $\vv s \in $Col$(A)$. Let's try to understand when this might be true by computing a basis for Col$(A)$.

From [](#inc_A), we notice that 
1. columns 1, 2 and 3 are **not independent**: column 3 = column 2 - column 1.
2. columns 3, 4 and 5 are **not independent**: column 5 = column 4 - column 3 = column 4 - column2 + column 1

However, we have that columns 1, 2  and 4 are **independent**! Since we can represent columns 3 and 5 in terms of columns 1, 2  and 4; columns 1, 2 and 4 span Col$(A)$. We conclude that columns 1, 2 and 4 form a basis for Col$(A)$, and dim(Col$(A)) = 3$. 

Now, let's take a closer look at [](#network): edges $1, 2$ and $3$ form a **loop** in the graph, while edges $3, 4$ and $5$ form another **loop**. In contrast, edges $1, 2$ and $4$ form a tree, which has no loops! This tells us that the edges of any tree in our graph gives us independent columns! 

So we now check if $\vv s \in$Col$(A)$ using columns $1, 2 $ and $4$.
\begin{equation}
\label{col_A}
&\bm -1 & -1 & 0 \\ 1 & 0 & -1 \\ 0 & 1 & 0 \\ 0 & 0 & 1\em
\bm f_1 \\ f_2 \\ f_4\em &= \bm -s \\ 0 \\ 0 \\ s\em \\
\Rightarrow &-f_1 - f_2 &= -s \\
& f_1 - f_4 &= 0 \\
& f_2 &= 0\\
& f_4 &= s
\end{equation}
which has solution $f_1 = f_4 = s$ and $f_2 = 0$. Therefore, one possible network flow is given by
$$f^* = \bm s \\ 0 \\ 0 \\ s \\ 0\em.$$
This corresponds to putting the flow on edges $1$ and $4$ as shown below. 
:::{figure}../figures/03-network_flow.jpg
:label:network_flow
:alt: Network Flow
:width: 350px
:align: center
:::

For any value of $s$, this solution can be represented in code using the following vector. Let's verify that it is a solution. 

In [38]:
s = 4
f_star = np.array([s, 0, 0, s, 0])

print(A@f_star)

[-4  0  0  4]


There are other ways to distribute the flow to satisfy $A \vv f = -\vv s$. That's where the null space of $A$ comes in!

Let's look at Null$A$. This is the solution set to $A \vv f = \vv 0$, which captures flow conservation in the absence of external sources. This corresponds to **(flow in) - (flow out) = 0** at each node: this is called _Kirchoff's current law in electric circuits_.

We already noticed that column 3= column 2 - column 1. Hence, one solution to $A \vv f = \vv 0$ is $\vv f_1 = \bm -1 \\ 1 \\ -1 \\ 0 \\ 0\em$ (verify for yourself), which corresponds to going around the $1 \to 3 \to 2$ loop! Similarly, column 5 = column 4- column 3, giving $\vv f_2 = \bm 0 \\ 0 \\ -1 \\ 1 \\ -1\em$, corresponding to the $3 \to 4 \to 5$ loop! $\vv f_1$ and $\vv f_2$ are linearly independent, and we know that 
$$
\textrm{dim(Null}(A)) = 5 \ (\textrm{our variables} \ \vv f \ \textrm{live in} \ \mathbb{R}^5)  - \textrm{dim(Col}(A)) = 5 - 3 = 2
$$
Hence, $\vv f_1$ and $\vv f_2$ form a basis! 

From [this theorem](./035-kernel_image.ipynb#soln_thm), we can therefore write the general solution to $A \vv f = -\vv s$ as
\begin{equation}
\label{gen_soln}
\vv f = \vv f^* + c_1 \vv f_1 + c_2 \vv f_2.
\end{equation}
The elements $\vv n \in $Null$(A)$ are called _circulations_ (why?).

Using this expression for the null space, any network flow for the network in [](#network) may be represented in code for arbitrary $c_1$ and $c_2$ as follows.

In [39]:
f_1 = np.array([-1, 1, -1, 0, 0])
f_2 = np.array([0, 0, -1, 1, -1])

c = np.array([1, 4])
f = f_star + c[0]*f_1 + c[1]*f_2
print(f)

[ 3  1 -5  8 -4]


## The Voltage Law


Now, instead of flows, let's discuss about potential differences, or voltages, across nodes.
:::{figure}../figures/03-network_volt.jpg
:label:network_volt
:alt: Network Voltage
:width: 350px
:align: center
:::
Solving $A^{\top} \vv x = \vv v$ tells us what potentials we need to put on the nodes to achieve the desired voltages. For example, the first row of $A^{\top} \vv x = \vv v$ is $-x_1 + x_2 = v_1$. This is _Kirchoff's Voltage Law_!

Let's start with Null$(A^{\top})$, which we find by setting $\vv v = \vv 0$.
\begin{equation}
\label{volt_eqn}
A^{\top} \vv x = \vv 0 \Rightarrow x_1 = x_2 \ (\textrm{first row}), \ x_1 = x_3 \ (\textrm{second row}), \ x_2 = x_4 \ (\textrm{fourth row}) \Rightarrow x_1 = x_2 = x_3 = x_4 = c.
\end{equation}
From [](#volt_eqn), Null$(A^{\top})$ is a line in $\mathbb{R}^4$ spanned by $\vv 1 = \bm 1 \\ 1 \\ 1 \\ 1\em$. The rank of $A$ must be $4 - 1 = 3$, which we saw was true above! This implies that we can increase the voltage across all nodes by the same amount, and achieve the same voltage drop between nodes. Makes sense!

The row space of $A$ is the column space of $A^{\top}$. There must be $3$ independent columns of $A^{\top}$ (since rank = 3). So, let's try to find them by inspection. The first three columns of $A^{\top}$ are
\begin{equation}
\label{volt_col}
\vv v_1 = \bm -1 \\ -1 \\ 0 \\ 0 \\ 0 \em, \ 
\vv v_2 = \bm 1 \\ 0 \\ -1 \\ -1 \\ 0 \em, \
\vv v_3 = \bm 0 \\ 1 \\ 1 \\ 0 \\ -1 \em
\end{equation}
which can be verified to to be linearly independent. Therefore, only voltage configurations $\vv v$ lying in span$\{\vv v_1, \vv v_2, \vv v_3\}$ can be encoded on this [graph](#network_volt). Therefore, for any $x_1$, $x_2$, and $x_3$, voltage configurations $x_1 \vv v_1 + x_2 \vv v_2 + x_3 \vv v_3$ are valid. Such configurations can be represented in code as follows. 


In [41]:
v_1 = np.array([-1, -1, 0, 0 ,0])
v_2 = np.array([1, 0, -1, -1, 0])
v_3 = np.array([0, 1, 1, 0, -1])
x = np.array([.5, 3, 1])

v = x[0]*v_1 + x[1]*v_2 + x[2]*v_3
print(v)

[ 2.5  0.5 -2.  -3.  -1. ]


:::{exercise} Can you interpret what the below statement means physically?
:label:challenge
$$
\vv v \in \textrm{Col}(A^{\top}) \ \textrm{if and only if} \ \vv f_1^{\top} \vv v = \vv 0 \ \textrm{and} \ \vv f_2^{\top} \vv v = \vv 0,
$$
where $\vv f_1$ and $\vv f_2$ are the basis elements for Null$(A)$?

```{solution} challenge
:class: dropdown 
The basis elements $\vv f_1$ and $\vv f_2$ encode _loops_ in the graph. This says that $\vv v$ is a valid voltage profile if and only if summing voltages along a loop equals zero. This is another way of stating Kirchoff's Voltage law. 
:::

We will understand where the statement in [](#challenge) comes from in the next couple of lectures! 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/01_Ch_2_Vector_Spaces_and_Bases/037-network.ipynb)


Using the current and voltage laws above, let's try to compute the currents in a circuit consisting of a current source and several resistors. In particular, we will consider the below circuit:
```{image} circuit_diagram.png
:alt: circuit_diagram
:width: 350px
:align: center
```
We can verify that the above circuit conforms to the network flow depicted in [](#network) by letting $I_S = s$, and $I_1, I_2, I_3, I_4, I_5 = f_1, f_2, f_3, f_4, f_5$. By the current law, we know that the currents must satisfy 
$$
\mathbf{i} = \bm I_1 \\ I_2 \\ I_3 \\ I_4 \\ I_5 \em = \mathbf{f}_\star + c_1 \mathbf{f}_1 + c_2 \mathbf{f}_2,
$$
for some numbers $c_1$ and $c_2$. Ohm's law allows us to relate the current between two nodes and the voltage drop between two nodes as $V_i = I_i R_i $, where $R_i$ is the resistance of edge $i$. We may therefore write
$$
\mathbf{v} = \bm V_1 \\ V_2 \\ V_3 \\ V_4 \\ V_5 \em = \bm R_1 I_1 \\ R_2 I_2 \\ R_3 I_3 \\ R_4 I_4 \\ R_5 I_5 \em = R \mathbf{i},
$$
where $R = \mathsf{diag}(R_1, R_2, R_3, R_4, R_5)$.

By the voltage law, $V_1, V_2, V_3, V_4, V_5$ is a consistent profile only if there exists some $x_1, x_2, x_3$ such that 
$$
    x_1 \mathbf{v}_1 + x_2 \mathbf{v}_2 + x_3 \mathbf{v}_3 = \mathbf{v} = R \mathbf{i} = R (\mathbf{f}_\star + c_1 \mathbf{f}_1 + c_2 \mathbf{f}_2). 
$$
Moving the terms involving $c_1$ and $c_2$ to the left side, we find that we must have
$$
    x_1 \mathbf{v}_1 + x_2 \mathbf{v}_2 + x_3 \mathbf{v}_3 - c_1 R \mathbf{f}_1 - c_2 R \mathbf{f}_2  = R \mathbf{i} = R \mathbf{f}_\star 
$$
The left hand side may be expressed as a matrix, leading to the equation
$$
    \bm \vv v_1 & \vv v_2 & \vv v_3 & -R \vv f_1 & -R \vv f_2 \em \bm x_1 \\ x_2 \\ x_3 \\ c_1 \\ c_2 \em = R \mathbf{f}_\star. 
$$

We may verify that for positive $R_1, R_2, R_3, R_4, R_5$, the vectors $\vv v_1$, $\vv v_2$, $\vv v_3$, $R \vv f_1$, $R \vv f_2$ are linearly independent. Therefore, we may invert the matrix $\bm \vv v_1 & \vv v_2 & \vv v_3 & -R \vv f_1 & -R \vv f_2 \em$ to solve the above equation, from which we find that 
$$ 
    \bm x_1 \\ x_2 \\ x_3 \\ c_1 \\ c_2 \em = \bm \vv v_1 & \vv v_2 & \vv v_3 & R \vv f_1 & R \vv f_2 \em^{-1} R \mathbf{f}_\star. 
$$

Let's try out some different resistances and check that the resulting current profiles make sense. 

In [35]:
R = np.diag([10, 10, 10, 10, 10])

stacked_matrix = np.stack([v_1, v_2, v_3, -R@f_1, -R@f_2], axis=1)
solution = np.linalg.inv(stacked_matrix)@R@f_star

x_1 = solution[0]; x_2 = solution[1]; x_3 = solution[2]; c_1 = solution[3]; c_2 = solution[4]

currents = f_star + c_1*f_1 + c_2*f_2
voltages = R@currents

print('currents: ', currents)
print('voltages:' , voltages)

currents:  [2.0000000e+00 2.0000000e+00 4.4408921e-16 2.0000000e+00 2.0000000e+00]
voltages: [2.0000000e+01 2.0000000e+01 4.4408921e-15 2.0000000e+01 2.0000000e+01]


Does the above solution make sense? When all resistances are equal, the two branches of the circuit are equally appealing, so the current is split evenly. Let's try a different resistance profile.

In [42]:
R = np.diag([100, 10, 10, 10, 10])

stacked_matrix = np.stack([v_1, v_2, v_3, -R@f_1, -R@f_2], axis=1)
solution = np.linalg.inv(stacked_matrix)@R@f_star

x_1 = solution[0]; x_2 = solution[1]; x_3 = solution[2]; c_1 = solution[3]; c_2 = solution[4]

currents = f_star + c_1*f_1 + c_2*f_2
voltages = R@currents

print('currents: ', currents)
print('voltages:' , voltages)

currents:  [ 0.45714286  3.54285714 -1.02857143  1.48571429  2.51428571]
voltages: [ 45.71428571  35.42857143 -10.28571429  14.85714286  25.14285714]


Now that the resistance $R_1$ is higher, the second path is prefereable. Therefore, a higher portion of the current flows through the second path. After bypassing the high resistance, some of the current reverts to the first path by flowing from node three to node two. 